In [3]:
from transformers import pipeline
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json

In [11]:
import torch

print(torch.cuda.is_available())
#print the cpu and gpu name
# print(torch.cuda.get_device_name(0))

# print(torch.version.cuda)
print("this cell run at: ",datetime.datetime.now())


False
this cell run at:  2023-05-12 10:07:24.171293


In [2]:
classifier = pipeline('sentiment-analysis')
res=classifier("I love you")
print(res)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

c:\Users\matan.s\Anaconda3b\envs\base2\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\matan.s\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9998656511306763}]


In [3]:
gen=pipeline('text-generation',model='distilgpt2')
res=gen('tell me about steve jobs')
print(res)

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/353M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
c:\Users\matan.s\Anaconda3b\envs\base2\lib\site-packages\transformers\generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 50 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'tell me about steve jobs as a whole.'}]


In [4]:
print(gen('tell me about steve jobs',max_length=100,num_return_sequences=5))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "tell me about steve jobs. The only people to want to know for sure who was behind this... well maybe that’s why I've decided to join a union and not be a part of this project for another month. As I said before, the best place to start is to be interested in your views. Be honest, I have absolutely no interest in what you think in this project. The only people to want to know for sure who was behind this... well maybe that’"}, {'generated_text': 'tell me about steve jobs. I told him on Twitter that his idea was worth it -- I don\'t know for sure what they were thinking....\n\n\nHere\'s another example of a problem with free speech: A free speechist who claims that it\'s "an essential part of the job market," claims he could only find "someplace" for the job market in the midtown Manhattan city under the guise of a free speech-friendly government. A free speechist who believes that'}, {'generated_text': 'tell me about steve jobs.›\nIn an interview with the Sun and o

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,AutoModelForSequenceClassification

In [6]:
classifier = pipeline('sentiment-analysis',model='distilbert-base-uncased-finetuned-sst-2-english')
res=classifier("I love you")
print(res)

[{'label': 'POSITIVE', 'score': 0.9998656511306763}]


In [7]:
model_name='distilbert-base-uncased-finetuned-sst-2-english'
model=AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer=AutoTokenizer.from_pretrained(model_name)

In [8]:
sequence="I love you"
res=tokenizer(sequence,return_tensors='pt')
print(res)
tokens=tokenizer.tokenize(sequence)
print(tokens)
ids=tokenizer.convert_tokens_to_ids(tokens)
print(ids)
decoded_string=tokenizer.decode(ids)
print(decoded_string)

{'input_ids': tensor([[ 101, 1045, 2293, 2017,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}
['i', 'love', 'you']
[1045, 2293, 2017]
i love you


In [9]:
import torch
import torch.nn.functional as F

X_train=['I love you','I hate you']
y_train=[1,0]
res=classifier(X_train)
print(res)

[{'label': 'POSITIVE', 'score': 0.9998656511306763}, {'label': 'NEGATIVE', 'score': 0.9991129040718079}]


In [10]:
batch=tokenizer(X_train,padding=True,truncation=True,return_tensors='pt')
print(batch)

{'input_ids': tensor([[ 101, 1045, 2293, 2017,  102],
        [ 101, 1045, 5223, 2017,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]])}


In [11]:
with torch.no_grad():
    logits=model(**batch).logits
    print(logits)
    print(F.softmax(logits,dim=1))
    labels=torch.argmax(F.softmax(logits,dim=1),dim=1)
    print(labels)

tensor([[-4.2756,  4.6393],
        [ 3.8724, -3.1543]])
tensor([[1.3436e-04, 9.9987e-01],
        [9.9911e-01, 8.8707e-04]])
tensor([1, 0])
